* rerun simulation high quality simulation for optimization trajectories
* establish theoretical justification for Degeneracy and Insensitivity

In [1]:
import pickle,scqubits
from pandas import concat
from numpy import pi,log10,isinf,isnan
from scipy.stats import pearsonr
from DiSuQ.utils import plotCompare,plotScatter

### data loading

In [2]:
def aggregate(Sim):
    Agg = []
    for index,sample in enumerate(Sim):
        metric = sample[0]
        path = sample[2].iloc[1:].reset_index()
        sample = concat([metric,path],axis=1,join='inner').drop(['index'],axis=1)
        sample['sample'] = index
        Agg.append(sample)
    Agg = concat(Agg).reset_index()
    Agg['delta'] *= -1
    #Agg['loc'] = Agg['sample'].apply(lambda circ:location[circ])
    Agg['EcS'] = 1/(1/Agg['Cx'] + 1/Agg['CJx'])
    return Agg

In [3]:
with open(r"./indPiDegeneracy.pickle", "rb") as input_file:
    PiDeg = aggregate(pickle.load(input_file))

In [4]:
with open(r"./indPiInsensitivity.pickle", "rb") as input_file:
    PiIns = aggregate(pickle.load(input_file))

In [5]:
with open(r"./simPi.pickle", "rb") as input_file:
    PiSim = aggregate(pickle.load(input_file))

In [6]:
with open(r"./indZeroDegeneracy.pickle", "rb") as input_file:
    ZeroDeg = aggregate(pickle.load(input_file))

In [7]:
with open(r"./indZeroInsensitivity.pickle", "rb") as input_file:
    ZeroIns = aggregate(pickle.load(input_file))

In [8]:
with open(r"./indZeroInsensitivity.pickle", "rb") as input_file:
    ZeroIns = aggregate(pickle.load(input_file))

In [9]:
with open(r"./simZero.pickle", "rb") as input_file:
    ZeroSim = aggregate(pickle.load(input_file))

In [10]:
with open(r"./simZeroThird.pickle", "rb") as input_file:
    ZeroThird = aggregate(pickle.load(input_file))

In [11]:
with open(r"./simZeroSub.pickle", "rb") as input_file:
    ZeroSub = aggregate(pickle.load(input_file))

In [12]:
with open(r"./simZeroSubIns.pickle", "rb") as input_file:
    ZeroSubIns = aggregate(pickle.load(input_file))

### sc Qubit simulation

In [13]:
def representationCapacity(El,Ej,Ec,EcJ):
    basis = {'Chi':7,'Theta':64,'Phi':1024}
    # theta capacity is sufficient for the given parameter range
    basis = {'Chi':7,'Theta':12,'Phi':1024}
    flux0 = 24*pi*2
    return flux0,basis

In [14]:
def HQSC(Agg,op=0):
    L = len(Agg)
    Degeneracy,Delta,E10,E20 = [],[],[],[]
    for index,circ in Agg.iterrows():
        El = circ['Lx']; Ej = circ['Jx']; EcJ = circ['CJx']; Ec = circ['Cx']
        EcS = 1/(1/Ec+1/EcJ)
        flux0,basis = representationCapacity(El,Ej,Ec,EcJ)
        flux_grid = scqubits.Grid1d(-flux0, flux0, basis['Phi'])

        zero_pi = scqubits.FullZeroPi(grid = flux_grid,
                           EJ   = Ej,
                           EL   = El,
                           ECJ  = EcJ, EC = None, ECS = EcS,
                           #EC  = 1/(1/EcS-1/EcJ),
                           ng   = 0, dEJ  = 0.0,
                             dCJ  = 0.0,
                             dEL  = 0.0,
                             dC   = 0.0,
                           flux = op,
                           ncut = basis['Theta'], zeropi_cutoff = 5,
                             zeta_cutoff = basis['Chi'])
        e0,e1,e2 = zero_pi.eigenvals(3)
        zero_pi.flux = op + .05
        n0,n1,n2 = zero_pi.eigenvals(3)
        D = log10((e2-e0)/(e1-e0))
        delta = log10((e1-e0)/abs((e1-e0)-(n1-n0)))
        Degeneracy.append(D); Delta.append(delta); E10.append(e1-e0); E20.append(e2-e0)
        if index%50 == 0:
            print(index,'/',L)
    Agg['D*'] = Degeneracy ; Agg['delta*'] = Delta ; Agg['E10*'] = E10 ; Agg['E20*'] = E20
    return Agg

### recalculation

In [ ]:
PiIns = HQSC(PiIns,.5)
PiDeg = HQSC(PiDeg,.5)
PiSim = HQSC(PiSim,.5)
ZeroDeg = HQSC(ZeroDeg,0)
ZeroIns = HQSC(ZeroIns,0)
ZeroSim = HQSC(ZeroSim,0)
ZeroSub = HQSC(ZeroSub,0)
ZeroSubIns = HQSC(ZeroSubIns,0)

0 / 220


### ratio-metric correlation plots

In [ ]:
Agg = PiIns

In [ ]:
x = log10(Agg['CJx']/Agg['EcS'])
x = log10(Agg['Jx']/Agg['EcS'])
x = log10(Agg['CJx']/Agg['Lx'])
#x = log10(Agg['Jx']/Agg['Lx'])
#y = Agg['D*']
y = Agg['delta*']
r,p = pearsonr(x[~isinf(y)],y[~isinf(y)])
print(r)

In [ ]:
plotScatter(log10(Agg['CJx']/Agg['EcS']),{'D':Agg['D'],'Delta':Agg['delta']},x_label='log(EcJ/EcS)',dot_size=5,
            size=(500,500),legend=True)
plotScatter(log10(Agg['Jx']/Agg['EcS']),{'D':Agg['D*'],'Delta':Agg['delta*']},x_label='log(Ej/EcS)',dot_size=5,
            size=(500,500),legend=True)
plotScatter(log10(Agg['CJx']/Agg['Lx']),{'D':Agg['D*'],'Delta':Agg['delta*']},x_label='log(EcJ/El)',dot_size=5,
            size=(500,500),legend=True)
plotScatter(log10(Agg['Jx']/Agg['Lx']),{'D':Agg['D*'],'Delta':Agg['delta*']},x_label='log(Ej/El)',dot_size=5,
            size=(500,500),legend=True)

## learning Zero-Pi degeneracy/insensitivity manifold
* optimization trajectories are geodesics over the manifold
$$(El,Ec,Ej,EcJ) \equiv R^4 \rightarrow R^2 \equiv (D,\Delta)$$
* data points are derived from optimization trajectories
* Learning incorporate entanglement and quantum physics of Zero-Pi 